# Анализ базы данных книжного магазина

## Описание проекта
В крупном сервисе для чтения книг по подписке необходимо провести анализ базы данных с книгами, издательствами, авторами, а также пользовательскими обзорами книг. Это необходимо для формирования ценностного предложения для нового продукта.

## Описание данных
**Таблица `books`** содержит данные о книгах:
* `book_id` — идентификатор книги;
* `author_id` — идентификатор автора;
* `title` — название книги;
* `num_pages` — количество страниц;
* `publication_date` — дата публикации книги;
* `publisher_id` — идентификатор издателя.  

**Таблица `authors`** содержит данные об авторах:
* `author_id` — идентификатор автора;
* `author` — имя автора.  

**Таблица `publishers`** содержит данные об издательствах:
* `publisher_id` — идентификатор издательства;
* `publisher` — название издательства.  

**Таблица `ratings`** содержит данные о пользовательских оценках книг:
* `rating_id` — идентификатор оценки;
* `book_id` — идентификатор книги;
* `username` — имя пользователя, оставившего оценку;
* `rating` — оценка книги.  

**Таблица `reviews`** содержит данные о пользовательских обзорах:
* `review_id` — идентификатор обзора;
* `book_id` — идентификатор книги;
* `username` — имя автора обзора;
* `text` — текст обзора.

## Схема данных
![](https://pictures.s3.yandex.net/resources/scheme_1589269096.png)

## Задания
- [x] Посчитайте, сколько книг вышло после 1 января 2000 года;
- [x] Для каждой книги посчитайте количество обзоров и среднюю оценку;
- [x] Определите издательство, которое выпустило наибольшее число книг толще 50 страниц — так вы исключите из анализа брошюры;
- [x] Определите автора с самой высокой средней оценкой книг — учитывайте только книги с 50 и более оценками;
- [x] Посчитайте среднее количество обзоров от пользователей, которые поставили больше 50 оценок.

In [1]:
# импортируем библиотеки
import pandas as pd
from sqlalchemy import create_engine
# устанавливаем параметры
db_config = {'user': 'praktikum_student', # имя пользователя
             'pwd': 'Sdf4$2;d-d30pp', # пароль
             'host': 'rc1b-wcoijxj3yxfsf3fs.mdb.yandexcloud.net',
             'port': 6432, # порт подключения
             'db': 'data-analyst-final-project-db'} # название базы данных
connection_string = 'postgresql://{}:{}@{}:{}/{}'.format(db_config['user'],
                                                         db_config['pwd'],
                                                         db_config['host'],
                                                         db_config['port'],
                                                         db_config['db'])
# сохраняем коннектор
engine = create_engine(connection_string, connect_args={'sslmode':'require'})

In [2]:
# вывод первых 5 строк таблиц
dbs = ['books', 'authors', 'ratings', 'reviews', 'publishers']

[display(pd.io.sql.read_sql(''' SELECT * FROM {} LIMIT 5'''.format(i), con = engine)) for i in dbs]


,book_id,author_id,title,num_pages,publication_date,publisher_id
0,1,546,'Salem's Lot,594,2005-11-01,93
1,2,465,1 000 Places to See Before You Die,992,2003-05-22,336
2,3,407,13 Little Blue Envelopes (Little Blue Envelope...,322,2010-12-21,135
3,4,82,1491: New Revelations of the Americas Before C...,541,2006-10-10,309
4,5,125,1776,386,2006-07-04,268


,author_id,author
0,1,A.S. Byatt
1,2,Aesop/Laura Harris/Laura Gibbs
2,3,Agatha Christie
3,4,Alan Brennert
4,5,Alan Moore/David Lloyd


,rating_id,book_id,username,rating
0,1,1,ryanfranco,4
1,2,1,grantpatricia,2
2,3,1,brandtandrea,5
3,4,2,lorichen,3
4,5,2,mariokeller,2


,review_id,book_id,username,text
0,1,1,brandtandrea,Mention society tell send professor analysis. ...
1,2,1,ryanfranco,Foot glass pretty audience hit themselves. Amo...
2,3,2,lorichen,Listen treat keep worry. Miss husband tax but ...
3,4,3,johnsonamanda,Finally month interesting blue could nature cu...
4,5,3,scotttamara,Nation purpose heavy give wait song will. List...


,publisher_id,publisher
0,1,Ace
1,2,Ace Book
2,3,Ace Books
3,4,Ace Hardcover
4,5,Addison Wesley Publishing Company


[None, None, None, None, None]

### Посчитайте, сколько книг вышло после 1 января 2000 года

In [3]:
query = ''' SELECT COUNT(*)
            FROM books
            WHERE publication_date > '2000-01-01'
        '''

pd.io.sql.read_sql(query, con = engine) 

,count
0,819


После 1 января 2000 года вышло 819 книг.

### Для каждой книги посчитайте количество обзоров и среднюю оценку

In [4]:
query = ''' SELECT b.title, AVG(ra.rating) AS avg_rating, COUNT(DISTINCT re.review_id) AS num_reviews
            FROM books AS b
            JOIN ratings AS ra ON b.book_id = ra.book_id
            JOIN reviews AS re ON b.book_id = re.book_id
            GROUP BY b.title, b.book_id
            ORDER BY num_reviews DESC, avg_rating DESC 
            LIMIT 10
        '''

pd.io.sql.read_sql(query, con = engine) 

,title,avg_rating,num_reviews
0,Twilight (Twilight #1),3.662500,7
1,Harry Potter and the Prisoner of Azkaban (Harr...,4.414634,6
2,Harry Potter and the Chamber of Secrets (Harry...,4.287500,6
3,The Book Thief,4.264151,6
4,The Glass Castle,4.206897,6
5,Outlander (Outlander #1),4.125000,6
6,The Hobbit or There and Back Again,4.125000,6
7,The Curious Incident of the Dog in the Night-Time,4.081081,6
8,The Lightning Thief (Percy Jackson and the Oly...,4.080645,6
9,Water for Elephants,3.977273,6


Сумерки - самая хайповая книга, однако это не делает её лучшей. Но какая книга лучшая, этот запрос не даёт сказать, потому что у большого количества книг с рейтингом 5 по несколько отзывов, что говорит о нерепрезентативности таких результатов.

### Определите издательство, которое выпустило наибольшее число книг толще 50 страниц — так вы исключите из анализа брошюры

In [5]:
query = ''' SELECT p.publisher, COUNT(b.book_id) AS num_books
            FROM books AS b 
            JOIN publishers AS p ON b.publisher_id = p.publisher_id 
            WHERE b.num_pages > 50
            GROUP BY p.publisher
            ORDER BY num_books DESC
            LIMIT 1
        '''

pd.io.sql.read_sql(query, con = engine) 

,publisher,num_books
0,Penguin Books,42


Похоже, что `Penguin Books` - самое достойное издание. Они выпустили 42 полноценных книги.

### Определите автора с самой высокой средней оценкой книг — учитывайте только книги с 50 и более оценками

In [6]:
query = ''' SELECT x.author AS author, AVG(x.avg_rating) AS avg_rating
            FROM (SELECT a.author AS author, 
                    b.book_id AS id, 
                    AVG(ra.rating) AS avg_rating,
                    COUNT(ra.rating)
                    FROM authors AS a
                    JOIN books AS b ON b.author_id = a.author_id
                    JOIN ratings AS ra ON ra.book_id = b.book_id
                    GROUP BY author, id
                    HAVING COUNT(ra.rating) >= 50) AS x
            GROUP BY author
            ORDER BY avg_rating DESC
            LIMIT 1
        '''

pd.io.sql.read_sql(query, con = engine) 

,author,avg_rating
0,J.K. Rowling/Mary GrandPré,4.283844


Тандем Джоан Роулинг и Мари ГрандПре позволил создать настоящую сказку о Мальчике, который выжил, которая тронула сердца миллионов детей и взрослых по всему миру. И рейтинги это только подтверждают.

### Посчитайте среднее количество обзоров от пользователей, которые поставили больше 50 оценок

In [7]:
query = ''' SELECT AVG(num_reviews) AS avg_num_reviews
            FROM (SELECT username, COUNT(text) AS num_reviews
                    FROM reviews AS re
                    FULL OUTER JOIN books AS b ON b.book_id = re.book_id
                    GROUP BY username) AS x
            JOIN (SELECT username, COUNT(rating) AS num_rating
                    FROM ratings
                    GROUP BY username
                    HAVING COUNT(rating) > 50) AS y ON x.username = y.username
        '''

pd.io.sql.read_sql(query, con = engine) 

,avg_num_reviews
0,24.333333


В среднем активный пользователь пишет 24.33 обзора. Действительно активный, не все за свою жизнь столько книг смогут осилить.

## Выводы
* У нас есть информация по самому активному издательству - `Penguin Books`. Это самый приоритетный партнёр.
* Запросы с самыми популярными книгами и самыми высоко оценёнными авторов позволяют нам сформировать нашу библиотеку начинная с этих книг и книг от этих авторов соответственно.
* Запрос с самыми популярными книгами показал, что в основном люди высоко оценивают книги в жанре фэнтези. Возможно, стоит сформировать подписку из книг этого жанра.
* У нас есть "зеро поинт" для оценки активных пользователей. Почему бы не подумать о поощрении активности со стороны пользователей и оформить скидку на подписку на следующий период?